# Feature filtering based on Mutual Information for classification

In [21]:
import pandas as pd
import os
import numpy as np
from sklearn.feature_selection import mutual_info_classif

# depending on the OS the path to the data file is different
if os.name == 'nt':
    data_nutr = pd.read_excel(r'..\data\nutrient-file-release2-jan22.xlsx', sheet_name='All solids & liquids per 100g')
elif os.name == 'posix':
    data_nutr = pd.read_excel(r'../data/nutrient-file-release2-jan22.xlsx', sheet_name='All solids & liquids per 100g')

# dataframe with only data columns


# print first 5 results.
data_nutr.head()

,Public Food Key,Classification,Food Name,"Energy with dietary fibre, equated \n(kJ)","Energy, without dietary fibre, equated \n(kJ)",Moisture (water) \n(g),Protein \n(g),Nitrogen \n(g),"Fat, total \n(g)",Ash \n(g),...,Leucine \n(mg),Lysine \n(mg),Methionine \n(mg),Phenylalanine \n(mg),Proline \n(mg),Serine \n(mg),Threonine \n(mg),Tyrosine \n(mg),Tryptophan \n(mg),Valine \n(mg)
0,F002258,31302.0,"Cardamom seed, dried, ground",1236,1012,8.3,10.8,1.72,6.7,5.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,155,NaN
1,F002893,31302.0,"Chilli (chili), dried, ground",1280,1002,10.8,13.4,2.14,14.3,11.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69,NaN
2,F002963,31302.0,"Cinnamon, dried, ground",1004,579,10.6,4.0,0.64,1.2,3.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49,NaN
3,F002970,31302.0,"Cloves, dried, ground",1389,1118,9.9,6.0,0.96,13.0,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,NaN
4,F003190,31302.0,"Coriander seed, dried, ground",1344,1009,8.9,12.4,1.98,17.8,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178,NaN


Since the dataset is largely continuous, we need to turn features into discrete ones.

## Variable discretisation

There are different methods to discretise continuous variables:

- **Equal-width binning**: divides the scope of possible values into N bins of the same width.
- **Equal-frequency binning**: divides the scope of possible values into N bins, each of them containing approximately the same number of samples.
- **Domain knowledge binning**: divides the scope of possible values into bins according to the domain knowledge.
<!-- There are other methods too listed below.

- ChiMerge: merges the bins using the Chi2 test to evaluate the statistical dependence of the classes and the feature.
- Entropy-based binning: merges the bins using the entropy of the classes and the feature.
- K-means binning: merges the bins using the K-means algorithm.
- Gaussian mixture binning: merges the bins using a Gaussian Mixture Model.
- Quantile binning: merges the bins so that each bin contains the same number of samples.
- Uniform binning: merges the bins so that each bin contains the same width.
- Recursive partitioning: merges the bins using a decision tree.
- Discretisation using decision trees: merges the bins using a decision tree.
- Discretisation using clustering: merges the bins using a clustering algorithm.
- Discretisation using support vector machines: merges the bins using a support vector machine.
- Discretisation using linear models: merges the bins using a linear model.
- Discretisation using nearest neighbours: merges the bins using a nearest neighbours algorithm.
- Discretisation using kernel density estimation: merges the bins using a kernel density estimation.
- Discretisation using fuzzy logic: merges the bins using a fuzzy logic algorithm.
- Discretisation using genetic algorithms: merges the bins using a genetic algorithm.
- Discretisation using simulated annealing: merges the bins using a simulated annealing algorithm.
- Discretisation using a neural network: merges the bins using a neural network.
- Discretisation using a random forest: merges the bins using a random forest.
- Discretisation using a linear discriminant analysis: merges the bins using a linear discriminant analysis.
- Discretisation using a quadratic discriminant analysis: merges the bins using a quadratic discriminant analysis.
- Discretisation using a principal component analysis: merges the bins using a principal component analysis.
- Discretisation using a factor analysis: merges the bins using a factor analysis.
- Discretisation using a canonical correlation analysis: merges the bins using a canonical correlation analysis.
- Discretisation using a partial least squares regression: merges the bins using a partial least squares regression.
- Discretisation using a ridge regression: merges the bins using a ridge regression. -->

WARNING: The choice of bins will influence the results of the mutual information filter.

In [32]:
# variable discretisation using pandas.qcut

# add new column with discretised values
data_nutr['Discretised Energy with dietary fibre, equated \n(kJ)'] = pd.cut(data_nutr['Energy with dietary fibre, equated \n(kJ)'], 20, labels=False)

# print the first few rows of the data for the two columns
data_nutr[['Energy with dietary fibre, equated \n(kJ)', 'Discretised Energy with dietary fibre, equated \n(kJ)']].head(10)

data_nutr.head()
# print the first few rows of the sorted data for the two columns
# data[['Energy with dietary fibre, equated \n(kJ)', 'Discretised Energy with dietary fibre, equated \n(kJ)']].sort_values(by='Energy with dietary fibre, equated \n(kJ)', ascending=False).head(10)

,Public Food Key,Classification,Food Name,"Energy with dietary fibre, equated \n(kJ)","Energy, without dietary fibre, equated \n(kJ)",Moisture (water) \n(g),Protein \n(g),Nitrogen \n(g),"Fat, total \n(g)",Ash \n(g),...,Lysine \n(mg),Methionine \n(mg),Phenylalanine \n(mg),Proline \n(mg),Serine \n(mg),Threonine \n(mg),Tyrosine \n(mg),Tryptophan \n(mg),Valine \n(mg),"Discretised Energy with dietary fibre, equated \n(kJ)"
0,F002258,31302.0,"Cardamom seed, dried, ground",15,12,2,9,9,11,19,...,9,0,0,0,0,0,0,10,0,15
1,F002893,31302.0,"Chilli (chili), dried, ground",15,12,2,10,10,15,19,...,9,0,0,0,0,0,0,7,0,15
2,F002963,31302.0,"Cinnamon, dried, ground",13,7,2,5,4,2,18,...,9,0,0,0,0,0,0,5,0,13
3,F002970,31302.0,"Cloves, dried, ground",16,14,2,7,7,14,19,...,9,0,0,0,0,0,0,3,0,16
4,F003190,31302.0,"Coriander seed, dried, ground",15,12,2,10,10,17,19,...,9,0,0,0,0,0,0,12,0,15


In [37]:
# Discretise all columns of data



ignored_columns = ['Public Food Key', 'Classification', 'Food Name']
label = 'Classification' # label to test
test_col = [] # names of columns
data_nutr = data_nutr.fillna(0) # having values of NaN prevents calculation of MI scores.

for nutrient in data_nutr.columns:
    if nutrient in ignored_columns: 
        continue # disregard first 3 columns ['Public Food Key', 'Classification', 'Food Name']
    else:
        test_col.append(nutrient) # for features below
        data_nutr[nutrient] = pd.cut(data_nutr[nutrient], 20, labels=False) # issues with pd.qcut relating to size of bins, proceeded with pd.cut
        # discretise each column so data is discrete and not continuous

# as follows in Week 9 Workshop - Feature filtering based on Mutual Information for classification
features = data_nutr[test_col]
features = features.fillna(0)
class_label = data_nutr[label]

data_nutr.head()

,Public Food Key,Classification,Food Name,"Energy with dietary fibre, equated \n(kJ)","Energy, without dietary fibre, equated \n(kJ)",Moisture (water) \n(g),Protein \n(g),Nitrogen \n(g),"Fat, total \n(g)",Ash \n(g),...,Lysine \n(mg),Methionine \n(mg),Phenylalanine \n(mg),Proline \n(mg),Serine \n(mg),Threonine \n(mg),Tyrosine \n(mg),Tryptophan \n(mg),Valine \n(mg),"Discretised Energy with dietary fibre, equated \n(kJ)"
0,F002258,31302.0,"Cardamom seed, dried, ground",15,12,1,9,9,11,19,...,9,0,0,0,0,0,0,9,0,15
1,F002893,31302.0,"Chilli (chili), dried, ground",15,12,1,9,9,15,19,...,9,0,0,0,0,0,0,7,0,15
2,F002963,31302.0,"Cinnamon, dried, ground",14,7,1,4,4,1,19,...,9,0,0,0,0,0,0,4,0,14
3,F002970,31302.0,"Cloves, dried, ground",17,15,1,7,7,15,19,...,9,0,0,0,0,0,0,2,0,17
4,F003190,31302.0,"Coriander seed, dried, ground",15,12,1,9,9,18,19,...,9,0,0,0,0,0,0,12,0,15


In [38]:


# Following in Week 9 Workshop - Feature filtering based on Mutual Information for classification
filtered_features = []
THRESHOLD = 0.2 # threshold value not fixed

mi_arr = mutual_info_classif(X=features, y=class_label, discrete_features=True)


for feature, mi in zip(features.columns, mi_arr):
    print(f'MI value for feature "{feature}": {mi:.4f}')

    if (mi >= THRESHOLD):
        filtered_features.append(feature)

print('\nFeature set after filtering with MI:', filtered_features)

MI value for feature "Energy with dietary fibre, equated 
(kJ)": 1.4810
MI value for feature "Energy, without dietary fibre, equated 
(kJ)": 1.4212
MI value for feature "Moisture (water) 
(g)": 1.5590
MI value for feature "Protein 
(g)": 1.4174
MI value for feature "Nitrogen 
(g)": 1.4118
MI value for feature "Fat, total 
(g)": 1.2292
MI value for feature "Ash 
(g)": 1.2711
MI value for feature "Total dietary fibre 
(g)": 0.9400
MI value for feature "Alcohol 
(g)": 0.0000
MI value for feature "Fructose 
(g)": 0.6420
MI value for feature "Glucose 
(g)": 0.6143
MI value for feature "Sucrose
(g)": 0.6634
MI value for feature "Maltose 
(g)": 0.0000
MI value for feature "Lactose 
(g)": 0.1737
MI value for feature "Galactose 
(g)": 0.0000
MI value for feature "Maltotrios 
(g)": 0.0000
MI value for feature "Total sugars (g)": 1.1657
MI value for feature "Added sugars (g)": 0.3399
MI value for feature "Free sugars 
(g)": 0.3337
MI value for feature "Starch 
(g)": 0.8690
MI value for feature "D